# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [26]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [27]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)
    print(len(file_path_list[0]))
 

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [28]:
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

907


In [29]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

797


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [30]:
# This should make a connection to a Cassandra instance your local machine 
# ('127.0.0.1')

from cassandra.cluster import Cluster
try:
    cluster=Cluster(['127.0.0.1'])
    session=cluster.connect()
    print("we are connected")
except Exception as e :
    print(" We have a problem connecting: {e}")
 

we are connected


#### Create Keyspace

In [31]:
keyspace_query = """CREATE KEYSPACE IF NOT EXISTS second_project 
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

 
try:
    session.execute(keyspace_query)
    print("keyspace created ok")
except Exception as e:
    print("having issue at keyspace : {e}")

keyspace created ok


#### Set Keyspace

In [32]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace("second_project")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




* Since we  query by sessionId and intemInSession we will have composite primary key when making table
* (sessionId ,itemInSession) will be theprimary key and we make the partioninng on sessionId

In [33]:
create_tableQ1="""CREATE TABLE IF NOT EXISTS session_item(artist text, song text, length float,
sessionId int, itemInSession int ,PRIMARY KEY (sessionId,itemInSession))"""
try:
    session.execute(create_tableQ1)
    print("Table Created with success")
except Exception as e:
    print("He have problem at creation of table :{e}")

Table Created with success


In [34]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
#here we read the data and insert it into our first  table that we need for query 1

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
       
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_item (artist,song,length,sessionId,itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query,(line[0], line[10], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [35]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query1="SELECT artist, song ,length from session_item where sessionId=338 and itemInSession=4"
result=session.execute(query1)
for r in result:
    print(r)

Row(artist='Faithless', song='50', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
* We need to use a composite key ((sessionId, userId),itemInSession) when creating a new  table for this  query.
* (sessionId, userId) represents the partition key and itemInSession represents  the clustering column.
* We need also to be able to sort data based on itemInSession.

                    

In [36]:
create_tableQ2="""CREATE TABLE IF NOT EXISTS user_session(sessionId int ,userId int , 
artist text, song text,  firstName text, lastName text,itemInSession int,
PRIMARY KEY ((sessionId,userId),itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)"""
try:
    session.execute(create_tableQ2)
    print(" Table  for query 2 created with success")
except Exception as e:
    print("There was the error : {e}")
source_file="event_datafile_new.csv"
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId, userId, artist, song, firstName, lastName, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])  ) )


 Table  for query 2 created with success


In [37]:
query2 = "SELECT artist, song, firstName, lastName FROM  user_session where sessionId = 182 and userId = 10"
try:
    response= session.execute(query2)
except Exception as e:
    print(e)

for r in response:
    print(r)


Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

* we need to have the following composite key : ((song),userId)
* song represents the partion key and since de dont whant duplicates we use userId which is unique as the cluseting column
                    

In [38]:

create_tableQ3 = """CREATE TABLE IF NOT EXISTS user_song (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"""

try: 
    session.execute(create_tableQ3)
    print("Scuccess creating  table for  query 3")
except Exception as e:
    print("Encountered the error : {e}")

Scuccess creating  table for  query 3


In [39]:
# reading and adding data 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s) "
        session.execute(query, (  line[9], int(line[10]), line[1], line[4] )  )

In [40]:
#runing the query
query3 = "SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'"
try:
    response = session.execute(query3)
except Exception as e:
    print(e)

for r in response:
    print(r)

Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [41]:
## TO-DO: Drop the table before closing out the sessions

In [42]:
session.execute("DROP TABLE IF EXISTS sparkify.session_item")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")
session.execute("DROP TABLE IF EXISTS sparkify.user_song")

### Close the session and cluster connection¶

In [43]:
session.shutdown()
cluster.shutdown()